In [68]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import json
import re
import lxml.html as lh
import datetime as dt

### Global confirmed corona cases


In [69]:
df = pd.read_csv('time_series_covid19_confirmed_global.csv')
df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,521,555,607,665,714,784,840,906,933,996
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,416,433,446,467,475,494,518,539,548,562
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,1761,1825,1914,1983,2070,2160,2268,2418,2534,2629
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,601,601,638,646,659,673,673,696,704,713
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,19,19,19,19,19,19,19,19,24,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Saint Pierre and Miquelon,France,46.885200,-56.315900,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
260,NaN,South Sudan,6.877000,31.307000,0,0,0,0,0,0,...,4,4,4,4,4,4,4,4,4,4
261,NaN,Western Sahara,24.215500,-12.885800,0,0,0,0,0,0,...,4,4,6,6,6,6,6,6,6,6
262,NaN,Sao Tome and Principe,0.186360,6.613081,0,0,0,0,0,0,...,4,4,4,4,4,4,4,4,4,4


In [70]:
df['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Diamond Princess', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary',
       'Iceland', 'India

In [71]:
data_EU_US = df[df['Country/Region'].isin(['Andorra', 'Albania', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina',
                                          'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 
                                          'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Latvia', 'Liechtenstein',
                                          'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands',
                                          'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania', 'Russia', 'San Marino',
                                          'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine',
                                          'United Kingdom', 'US', 'Venezuela'])]
data_EU_US

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,416,433,446,467,475,494,518,539,548,562
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,601,601,638,646,659,673,673,696,704,713
16,NaN,Austria,47.5162,14.5501,0,0,0,0,0,0,...,13555,13806,13945,14041,14226,14336,14476,14595,14671,14749
22,NaN,Belarus,53.7098,27.9534,0,0,0,0,0,0,...,1981,2226,2578,2919,3281,3728,4204,4779,4779,4779
23,NaN,Belgium,50.8333,4.0000,0,0,0,0,0,0,...,26667,28018,29647,30589,31119,33573,34809,36138,37183,38496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,British Virgin Islands,United Kingdom,18.4207,-64.6400,0,0,0,0,0,0,...,3,3,3,3,3,3,3,4,4,4
251,Turks and Caicos Islands,United Kingdom,21.6940,-71.7979,0,0,0,0,0,0,...,8,8,9,10,10,10,11,11,11,11
256,"Bonaire, Sint Eustatius and Saba",Netherlands,12.1784,-68.2385,0,0,0,0,0,0,...,2,2,3,3,3,3,3,3,3,5
258,Falkland Islands (Malvinas),United Kingdom,-51.7963,-59.5236,0,0,0,0,0,0,...,5,5,5,5,11,11,11,11,11,11


In [72]:
df_countries_only = data_EU_US[data_EU_US['Province/State'].isna()]
df_countries_only 

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,416,433,446,467,475,494,518,539,548,562
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,601,601,638,646,659,673,673,696,704,713
16,NaN,Austria,47.5162,14.5501,0,0,0,0,0,0,...,13555,13806,13945,14041,14226,14336,14476,14595,14671,14749
22,NaN,Belarus,53.7098,27.9534,0,0,0,0,0,0,...,1981,2226,2578,2919,3281,3728,4204,4779,4779,4779
23,NaN,Belgium,50.8333,4.0000,0,0,0,0,0,0,...,26667,28018,29647,30589,31119,33573,34809,36138,37183,38496
27,NaN,Bosnia and Herzegovina,43.9159,17.6791,0,0,0,0,0,0,...,901,946,1009,1037,1083,1110,1167,1214,1268,1285
30,NaN,Bulgaria,42.7339,25.4858,0,0,0,0,0,0,...,635,661,675,685,713,747,800,846,878,894
87,NaN,Croatia,45.1000,15.2000,0,0,0,0,0,0,...,1495,1534,1600,1650,1704,1741,1791,1814,1832,1871
90,NaN,Cyprus,35.1264,33.4299,0,0,0,0,0,0,...,595,616,633,662,695,715,735,750,761,767
91,NaN,Czechia,49.8175,15.4730,0,0,0,0,0,0,...,5732,5831,5991,6059,6111,6216,6433,6549,6606,6746


In [73]:
data_clean = df_countries_only.drop(['Province/State'], axis=1)
data_clean

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20
1,Albania,41.1533,20.1683,0,0,0,0,0,0,0,...,416,433,446,467,475,494,518,539,548,562
3,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,...,601,601,638,646,659,673,673,696,704,713
16,Austria,47.5162,14.5501,0,0,0,0,0,0,0,...,13555,13806,13945,14041,14226,14336,14476,14595,14671,14749
22,Belarus,53.7098,27.9534,0,0,0,0,0,0,0,...,1981,2226,2578,2919,3281,3728,4204,4779,4779,4779
23,Belgium,50.8333,4.0000,0,0,0,0,0,0,0,...,26667,28018,29647,30589,31119,33573,34809,36138,37183,38496
27,Bosnia and Herzegovina,43.9159,17.6791,0,0,0,0,0,0,0,...,901,946,1009,1037,1083,1110,1167,1214,1268,1285
30,Bulgaria,42.7339,25.4858,0,0,0,0,0,0,0,...,635,661,675,685,713,747,800,846,878,894
87,Croatia,45.1000,15.2000,0,0,0,0,0,0,0,...,1495,1534,1600,1650,1704,1741,1791,1814,1832,1871
90,Cyprus,35.1264,33.4299,0,0,0,0,0,0,0,...,595,616,633,662,695,715,735,750,761,767
91,Czechia,49.8175,15.4730,0,0,0,0,0,0,0,...,5732,5831,5991,6059,6111,6216,6433,6549,6606,6746


In [74]:
df_good = data_clean.reset_index()
del df_good['index']
df_good

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20
0,Albania,41.1533,20.1683,0,0,0,0,0,0,0,...,416,433,446,467,475,494,518,539,548,562
1,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,...,601,601,638,646,659,673,673,696,704,713
2,Austria,47.5162,14.5501,0,0,0,0,0,0,0,...,13555,13806,13945,14041,14226,14336,14476,14595,14671,14749
3,Belarus,53.7098,27.9534,0,0,0,0,0,0,0,...,1981,2226,2578,2919,3281,3728,4204,4779,4779,4779
4,Belgium,50.8333,4.0000,0,0,0,0,0,0,0,...,26667,28018,29647,30589,31119,33573,34809,36138,37183,38496
5,Bosnia and Herzegovina,43.9159,17.6791,0,0,0,0,0,0,0,...,901,946,1009,1037,1083,1110,1167,1214,1268,1285
6,Bulgaria,42.7339,25.4858,0,0,0,0,0,0,0,...,635,661,675,685,713,747,800,846,878,894
7,Croatia,45.1000,15.2000,0,0,0,0,0,0,0,...,1495,1534,1600,1650,1704,1741,1791,1814,1832,1871
8,Cyprus,35.1264,33.4299,0,0,0,0,0,0,0,...,595,616,633,662,695,715,735,750,761,767
9,Czechia,49.8175,15.4730,0,0,0,0,0,0,0,...,5732,5831,5991,6059,6111,6216,6433,6549,6606,6746


In [75]:
df_good.describe()

,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20
count,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,...,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000
mean,47.334819,11.869791,0.021277,0.021277,0.085106,0.106383,0.170213,0.191489,0.276596,0.319149,...,28283.425532,29677.021277,31516.063830,32689.574468,33925.893617,35293.617021,36980.659574,38453.936170,39735.659574,41148.978723
std,9.642851,25.575438,0.145865,0.145865,0.408059,0.520620,0.842335,0.850532,1.077479,1.162949,...,79314.850361,83652.802168,88291.686305,91943.668013,95840.970333,100010.923520,104816.685295,109490.777838,113999.502242,118058.402927
min,6.423800,-95.712900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,79.000000,79.000000,79.000000,79.000000,79.000000,79.000000,79.000000,79.000000,79.000000,81.000000
25%,42.620100,6.773150,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,713.000000,744.000000,785.000000,811.500000,871.500000,918.500000,1029.000000,1083.000000,1129.500000,1184.000000
50%,47.140000,15.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2203.000000,2511.000000,2777.000000,3064.000000,3281.000000,3373.000000,3444.000000,3489.000000,3681.000000,3783.000000
75%,52.637500,22.852800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,12736.000000,13695.000000,14857.500000,15487.500000,15837.000000,16213.500000,16658.500000,16808.500000,17221.500000,17728.500000
max,64.963100,90.000000,1.000000,1.000000,2.000000,3.000000,5.000000,5.000000,5.000000,5.000000,...,496535.000000,526396.000000,555313.000000,580619.000000,607670.000000,636350.000000,667592.000000,699706.000000,732197.000000,759086.000000


### Global Deaths corona


In [76]:
df_d = pd.read_csv('time_series_covid19_deaths_global.csv')
df_d

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,15,18,18,21,23,25,30,30,30,33
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,23,23,23,23,24,25,26,26,26,26
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,256,275,293,313,326,336,348,364,367,375
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,26,26,29,29,31,33,33,35,35,36
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Saint Pierre and Miquelon,France,46.885200,-56.315900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
260,NaN,South Sudan,6.877000,31.307000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
261,NaN,Western Sahara,24.215500,-12.885800,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
262,NaN,Sao Tome and Principe,0.186360,6.613081,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
data_EU_US_d = df_d[df_d['Country/Region'].isin(['Andorra', 'Albania', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina',
                                          'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 
                                          'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Latvia', 'Liechtenstein',
                                          'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands',
                                          'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania', 'Russia', 'San Marino',
                                          'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine',
                                          'United Kingdom', 'US', 'Venezuela'])]
data_EU_US_d

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,23,23,23,23,24,25,26,26,26,26
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,26,26,29,29,31,33,33,35,35,36
16,NaN,Austria,47.5162,14.5501,0,0,0,0,0,0,...,319,337,350,368,384,393,410,431,443,452
22,NaN,Belarus,53.7098,27.9534,0,0,0,0,0,0,...,19,23,26,29,33,36,40,42,45,47
23,NaN,Belgium,50.8333,4.0000,0,0,0,0,0,0,...,3019,3346,3600,3903,4157,4440,4857,5163,5453,5683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,British Virgin Islands,United Kingdom,18.4207,-64.6400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
251,Turks and Caicos Islands,United Kingdom,21.6940,-71.7979,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
256,"Bonaire, Sint Eustatius and Saba",Netherlands,12.1784,-68.2385,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
258,Falkland Islands (Malvinas),United Kingdom,-51.7963,-59.5236,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
df_countries_only_d = data_EU_US_d[data_EU_US_d['Province/State'].isna()]
df_countries_only_d 

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,23,23,23,23,24,25,26,26,26,26
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,26,26,29,29,31,33,33,35,35,36
16,NaN,Austria,47.5162,14.5501,0,0,0,0,0,0,...,319,337,350,368,384,393,410,431,443,452
22,NaN,Belarus,53.7098,27.9534,0,0,0,0,0,0,...,19,23,26,29,33,36,40,42,45,47
23,NaN,Belgium,50.8333,4.0000,0,0,0,0,0,0,...,3019,3346,3600,3903,4157,4440,4857,5163,5453,5683
27,NaN,Bosnia and Herzegovina,43.9159,17.6791,0,0,0,0,0,0,...,36,37,39,39,40,41,43,46,47,48
30,NaN,Bulgaria,42.7339,25.4858,0,0,0,0,0,0,...,25,28,29,32,35,36,38,41,41,42
87,NaN,Croatia,45.1000,15.2000,0,0,0,0,0,0,...,21,21,23,25,31,33,35,36,39,47
90,NaN,Cyprus,35.1264,33.4299,0,0,0,0,0,0,...,10,10,11,12,12,12,12,12,12,12
91,NaN,Czechia,49.8175,15.4730,0,0,0,0,0,0,...,119,129,138,143,161,166,169,173,181,186


In [79]:
data_clean_d = df_countries_only_d.drop(['Province/State'], axis=1)
data_clean_d

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20
1,Albania,41.1533,20.1683,0,0,0,0,0,0,0,...,23,23,23,23,24,25,26,26,26,26
3,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,...,26,26,29,29,31,33,33,35,35,36
16,Austria,47.5162,14.5501,0,0,0,0,0,0,0,...,319,337,350,368,384,393,410,431,443,452
22,Belarus,53.7098,27.9534,0,0,0,0,0,0,0,...,19,23,26,29,33,36,40,42,45,47
23,Belgium,50.8333,4.0000,0,0,0,0,0,0,0,...,3019,3346,3600,3903,4157,4440,4857,5163,5453,5683
27,Bosnia and Herzegovina,43.9159,17.6791,0,0,0,0,0,0,0,...,36,37,39,39,40,41,43,46,47,48
30,Bulgaria,42.7339,25.4858,0,0,0,0,0,0,0,...,25,28,29,32,35,36,38,41,41,42
87,Croatia,45.1000,15.2000,0,0,0,0,0,0,0,...,21,21,23,25,31,33,35,36,39,47
90,Cyprus,35.1264,33.4299,0,0,0,0,0,0,0,...,10,10,11,12,12,12,12,12,12,12
91,Czechia,49.8175,15.4730,0,0,0,0,0,0,0,...,119,129,138,143,161,166,169,173,181,186


In [85]:
df_good_d = data_clean_d.reset_index()
del df_good_d['index']
df_good_d
df_good_d = df_good_d.drop(columns=['Lat', 'Long']).set_index("Country/Region")
df_good_d = df_good_d.T.reset_index()
df_good_d = df_good_d.rename(columns={'index':'Date'})
df_good_d.Date = df_good_d.Date.astype('datetime64')
df_good_d

Country/Region,Date,Albania,Andorra,Austria,Belarus,Belgium,Bosnia and Herzegovina,Bulgaria,Croatia,Cyprus,...,Slovakia,Slovenia,Spain,Sweden,Switzerland,Turkey,Ukraine,United Kingdom,US,Venezuela
0,2020-01-22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-01-23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-01-24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-01-25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020-01-26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2020-04-15,25,33,393,36,4440,41,36,33,12,...,6,61,18708,1203,1239,1518,108,12868,28325,9
85,2020-04-16,26,33,410,40,4857,43,38,35,12,...,8,61,19315,1333,1281,1643,116,13729,32916,9
86,2020-04-17,26,35,431,42,5163,46,41,36,12,...,9,66,20002,1400,1327,1769,125,14576,36773,9
87,2020-04-18,26,35,443,45,5453,47,41,39,12,...,11,70,20043,1511,1368,1890,133,15464,38664,9


# trasposed Global Deaths

In [86]:
df_good_d = df_good_d.drop(columns=['Venezuela'], axis=1)
df_good_d

Country/Region,Date,Albania,Andorra,Austria,Belarus,Belgium,Bosnia and Herzegovina,Bulgaria,Croatia,Cyprus,...,Serbia,Slovakia,Slovenia,Spain,Sweden,Switzerland,Turkey,Ukraine,United Kingdom,US
0,2020-01-22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-01-23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-01-24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-01-25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020-01-26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2020-04-15,25,33,393,36,4440,41,36,33,12,...,99,6,61,18708,1203,1239,1518,108,12868,28325
85,2020-04-16,26,33,410,40,4857,43,38,35,12,...,103,8,61,19315,1333,1281,1643,116,13729,32916
86,2020-04-17,26,35,431,42,5163,46,41,36,12,...,110,9,66,20002,1400,1327,1769,125,14576,36773
87,2020-04-18,26,35,443,45,5453,47,41,39,12,...,117,11,70,20043,1511,1368,1890,133,15464,38664


# trasposed Global confirmed

In [87]:
df_good = data_clean.reset_index()
del df_good['index']
df_good = df_good.drop(columns=['Lat', 'Long']).set_index("Country/Region")
df_good = df_good.T.reset_index()
df_good = df_good.rename(columns={'index':'Date'})
df_good.Date = df_good.Date.astype('datetime64')
df_good = df_good.drop(columns=['Venezuela'], axis=1)
df_good

Country/Region,Date,Albania,Andorra,Austria,Belarus,Belgium,Bosnia and Herzegovina,Bulgaria,Croatia,Cyprus,...,Serbia,Slovakia,Slovenia,Spain,Sweden,Switzerland,Turkey,Ukraine,United Kingdom,US
0,2020-01-22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2020-01-23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2020-01-24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,2020-01-25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,2020-01-26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2020-04-15,494,673,14336,3728,33573,1110,747,1741,715,...,4873,863,1248,177644,11927,26336,69392,3764,98476,636350
85,2020-04-16,518,673,14476,4204,34809,1167,800,1791,735,...,5318,977,1268,184948,12540,26732,74193,4161,103093,667592
86,2020-04-17,539,696,14595,4779,36138,1214,846,1814,750,...,5690,1049,1304,190839,13216,27078,78546,4662,108692,699706
87,2020-04-18,548,704,14671,4779,37183,1268,878,1832,761,...,5994,1089,1317,191726,13822,27404,82329,5106,114217,732197
